In [1]:
# %load_ext autotime

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
# from mei import Size_Distribution_Optics
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [4]:
import glob
import dask.dataframe as dd

In [5]:
import sklearn
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.metrics import r2_score 
import joblib

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Activation

from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, ReduceLROnPlateau, EarlyStopping

2023-08-17 19:08:14.510381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
from kerastuner.tuners import RandomSearch, BayesianOptimization

In [8]:
def normalize(df):
    return (df - df.mean()) / df.std()

def normalize1(df):    
    mx = df.max()
    mn = df.min()
    
    mx['SSA'] = 1.0
    mx['Asym'] = 1.0
    
    mn['SSA'] = 0.0
    mn['Asym'] = 0.0
    
    new = (df - mn) / (mx - mn)    
    return new


def load_opt(filename):
    with open(filename, "r") as fp:
        data = json.load(fp)
    return data

In [9]:
columns = ['Extinction', 'Scattering', 'SSA', 'Asym', 'coating', 'core_dia', 'n_core', 'k_core', 'n_shell', 'k_shell', 'density', 'lambda']
df1 = pd.read_csv('/work/bb1070/b382177/mie/icon/12-05-2023/mie_x1.csv', index_col=0).sort_index().sort_index().iloc[:-1224, :]
df2 = pd.read_csv('/work/bb1070/b382177/mie/icon/12-05-2023/mie_x2.csv', index_col=0).sort_index().sort_index().iloc[:-1224, :]
df = pd.concat([df1, df2], axis=0).reset_index()
df = df[columns].dropna().astype('float')
df['x'] = np.pi * df['core_dia'] / df['lambda']
df

,Extinction,Scattering,SSA,Asym,coating,core_dia,n_core,k_core,n_shell,k_shell,density,lambda,x
0,2.371424,1.497649,0.631540,0.898342,0.0,4000.0,1.451990,0.022362,1.330928,4.128916e-08,2.6,700.0,17.951958
1,2.203139,1.330449,0.603888,0.882517,0.0,5000.0,1.453480,0.023396,1.327650,2.061950e-06,2.6,950.0,16.534698
2,2.173950,1.202943,0.553344,0.947042,0.0,9000.0,1.451020,0.018632,1.332398,9.574458e-09,2.6,600.0,47.123890
3,2.068879,1.123843,0.543213,0.952623,0.0,10000.0,1.483700,0.099080,1.389200,9.470000e-08,2.6,210.0,149.599650
4,2.150144,1.163233,0.541003,0.941646,0.0,10000.0,1.453000,0.022940,1.329120,3.110181e-07,2.6,850.0,36.959914
...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.184309,0.033168,0.179958,0.086705,0.4,700.0,1.394680,0.006191,1.420600,1.426000e-01,1.7,4600.0,0.478068
599996,0.181849,0.010640,0.058508,0.054755,0.4,900.0,1.364826,0.022669,1.276529,1.827067e-01,1.7,7100.0,0.398230
599997,0.265074,0.004225,0.015940,0.016758,0.4,3000.0,1.613500,0.648098,2.010000,6.500000e-01,1.7,50000.0,0.188496
599998,0.193847,0.001301,0.006709,0.009228,0.4,4000.0,1.613500,0.648098,2.020000,6.850000e-01,1.7,90000.0,0.139626


In [10]:
# df[["Extinction", "SSA", "Asym"]].to_csv('/work/bb1070/b382177/mie/icon/mie_out.txt', float_format='%.3f', sep=' ')
# df[['coating', 'x', 'n_core', 'k_core', 'n_shell', 'k_shell', 'lambda']].to_csv('/work/bb1070/b382177/mie/icon/mie_input.txt', float_format='%.3f', sep=' ')

In [11]:
mx = df.max()
mn = df.min()

pd.concat([pd.DataFrame(mx), pd.DataFrame(mn)], axis=1)

,0,0
Extinction,4.427469,3.949925e-07
Scattering,4.427461,2.459024e-15
SSA,1.000367,1.902386e-11
Asym,0.980657,1.920404e-08
coating,0.400000,0.000000e+00
core_dia,20000.000000,1.000000e+01
n_core,2.768000,1.066360e+00
k_core,1.002280,5.000000e-09
n_shell,2.094000,1.172000e+00
k_shell,0.730000,1.062778e-09


In [12]:
qt = QuantileTransformer(n_quantiles=1000, output_distribution="normal", random_state=0)
df_shuffled = df.sample(frac=1, random_state=40).reset_index(drop=True)
df_shuffled = normalize1(df_shuffled)

out_col = ["Extinction", "Scattering", "Asym"]
in_col = ['coating', 'x', 'n_core', 'k_core', 'n_shell', 'k_shell', 'lambda']

y = df_shuffled[out_col]
y[out_col] = qt.fit_transform(y)
x = df_shuffled[in_col]

split = 0.9
train_size = int(split * len(x))
train_x = x.iloc[:train_size, :].reset_index(drop=True)
train_y = y.iloc[:train_size, :].reset_index(drop=True)
test_x = x.iloc[train_size:, :].reset_index(drop=True)
test_y = y.iloc[train_size:, :].reset_index(drop=True)
train_y

,Extinction,Scattering,Asym
0,-0.385656,-0.134130,-0.108732
1,-0.825163,-0.511151,-0.509331
2,1.483993,1.338700,0.826924
3,-0.051317,-0.452069,-0.513292
4,0.518457,0.429787,1.835915
...,...,...,...
539995,-0.990207,-0.841703,-0.810697
539996,0.220158,0.192439,0.161772
539997,-0.956842,-5.199338,-1.599475
539998,-0.213032,-0.563891,-0.612526


In [13]:
def create_model(hp):
    model = Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(7,)))
    hp_optimizers = hp.Choice('optimizer', values=["adam", "sgd", 'rmsprop'])   
    hp_units = hp.Choice('units', [8, 16, 32, 64])
    activation = hp.Choice('activation', ['gelu', 'relu', 'sigmoid', 'tanh'])
    for i in range(hp.Int('num_layers', 1, 5)): 
        #hp_units = hp.Choice('units_' + str(i), [8, 16, 32, 64])
        #activation = hp.Choice('activation_' + str(i), ['gelu', 'relu', 'sigmoid', 'tanh'])
        model.add(Dense(units=hp_units, activation=activation))
        #model.add(Dropout(hp.Choice('dropout_'+ str(i), values=[0.0, 0.1, 0.2])))
        
    model.add(Dense(3, name="output_layer"))
    model.compile(loss="mean_absolute_error", optimizer=hp_optimizers, metrics=["mean_squared_error"])
    # model.compile(loss="mse", optimizer=hp_optimizers, metrics=['mse'])
    return model

tuner = BayesianOptimization(create_model, 
                             seed=1234,
                             objective="val_mean_squared_error", 
                             max_trials=50, 
                             distribution_strategy=tf.distribute.MirroredStrategy(),
                             directory='model',
                             overwrite=False,
                             project_name="mie")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Reloading Tuner from model/mie/tuner0.json


2023-08-17 19:08:25.112309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-17 19:08:25.112345: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-17 19:08:25.112364: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (l40003.lvt.dkrz.de): /proc/driver/nvidia/version does not exist
2023-08-17 19:08:25.112624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rms'], 'ordered': False}
units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32, 64], 'ordered': True}
activation (Choice)
{'default': 'gelu', 'conditions': [], 'values': ['gelu', 'relu', 'sigmoid', 'tanh'], 'ordered': False}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}


In [15]:
tuner.search(train_x, train_y, epochs=200, batch_size=250, validation_data=(test_x, test_y))

Trial 50 Complete [00h 07m 26s]
val_mean_squared_error: 0.04747132584452629

Best val_mean_squared_error So Far: 0.032091204077005386
Total elapsed time: 02h 43m 14s
INFO:tensorflow:Oracle triggered exit


In [16]:
tuner.results_summary()

Results summary
Results in model/mie
Showing 10 best trials
Objective(name="val_mean_squared_error", direction="min")

Trial 29 summary
Hyperparameters:
optimizer: adam
units: 64
activation: gelu
num_layers: 5
Score: 0.032091204077005386

Trial 27 summary
Hyperparameters:
optimizer: adam
units: 32
activation: tanh
num_layers: 3
Score: 0.03307025879621506

Trial 15 summary
Hyperparameters:
optimizer: adam
units: 64
activation: tanh
num_layers: 3
Score: 0.033729273825883865

Trial 37 summary
Hyperparameters:
optimizer: adam
units: 32
activation: gelu
num_layers: 4
Score: 0.03562641143798828

Trial 24 summary
Hyperparameters:
optimizer: adam
units: 64
activation: gelu
num_layers: 4
Score: 0.03632396087050438

Trial 23 summary
Hyperparameters:
optimizer: adam
units: 64
activation: relu
num_layers: 5
Score: 0.03884287551045418

Trial 39 summary
Hyperparameters:
optimizer: adam
units: 16
activation: tanh
num_layers: 3
Score: 0.040453314781188965

Trial 45 summary
Hyperparameters:
optimizer: 

In [17]:
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.values

{'optimizer': 'adam', 'units': 64, 'activation': 'gelu', 'num_layers': 5}